<a href="https://colab.research.google.com/github/massaro8/cnn-pytorch-example/blob/main/Introduzione_cnn_pytorch_empty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Introduzione alle CNN con PyTorch


## Obiettivi didattici:  
1. Capire cos'è una **convoluzione** (filtri, kernel, stride, padding) e perché è utile con le immagini.  
2. Comprendere il **pooling** e il concetto di invarianza locale.  
3. Costruire, addestrare, valutare e salvare una piccola **CNN** in PyTorch sul dataset **MNIST**.  
4. Usare strumenti pratici: scelta automatica del **device** (CPU/GPU), barra di progresso con `tqdm`, funzione di **accuratezza**.  
5. Migliorare la rete con **regolarizzazione** (dropout, L2/weight decay), **Batch Normalization** e **scheduler** del learning rate.  

## Architettura CNN

![Alt text](https://media.datacamp.com/cms/ad_4nxct55fjxboktz5ezpyzmmkc28dy6tk3s_djp9uljfjwigsm4oagqnrvbr-edpro2ggylzl4odhtbc3xapxf-y527snl-i_noynj1uteapbm_erw-hijzkvaqmt9oiap8__pp083.png)






## Prerequisiti e setup

Esegui la cella seguente per importare le librerie. In **Colab** molte sono già presenti.  

```bash

# Installazione pacchetti principali
pip install --upgrade torch torchvision torchaudio tqdm matplotlib scikit-learn
```


In [ ]:
! pip install --upgrade torch torchvision torchaudio tqdm matplotlib scikit-learn

In [ ]:

# Librerie standard Python
import os
import random
import math
import time


# Libreria per calcolo numerico e manipolazione di array multidimensionali
import numpy as np

# Libreria per grafici e visualizzazioni (curve di training, immagini, confusion matrix, ecc.)
import matplotlib.pyplot as plt

# Libreria principale per il Deep Learning
import torch


# Moduli principali di PyTorch
from torch import nn,optim


# Sotto-modulo con funzioni matematiche/attivazioni da usare direttamente (es. F.relu, F.softmax)
import torch.nn.functional as F


# Modulo per la gestione dei dataset
from torch.utils.data import DataLoader, random_split, Subset


# Libreria PyTorch per la Computer Vision (dataset, trasformazioni e modelli pre-addestrati)
import torchvision
from torchvision import datasets,transforms


# Libreria per barre di avanzamento eleganti e automatiche
from tqdm.auto import tqdm


# Stampa le versioni di PyTorch e Torchvision (utile per debug o compatibilità)
print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")


# Funzione per fissare i semi casuali (reproducibilità)
# Garantisce che ogni esecuzione produca gli stessi risultati
def set_seed(seed: int = 42):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

# Richiama la funzione di fissaggio del seed casuale
set_seed(seed = 42)

# Se è disponibile una GPU CUDA, la utilizza; altrimenti usa la CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Stampa quale dispositivo è stato selezionato
print("Device",device)


# 🧠 Basi delle CNN — Convoluzione (Filtri, Stride, Padding, Feature Map)

Le Convolutional Neural Networks (CNN) sono reti progettate per analizzare immagini e dati spaziali, sfruttando il concetto di convoluzione: un’operazione matematica che combina un’immagine d’ingresso con un piccolo filtro (o kernel) per estrarre pattern locali come bordi, angoli o texture.

## 🔹 1. Filtri (Kernels)

Un filtro (o kernel) è una piccola matrice di pesi (es. 3×3 o 5×5) che scorre sull’immagine originale (input).
A ogni posizione, il filtro e la finestra dell’immagine vengono moltiplicati elemento per elemento e poi sommati → ottenendo un singolo numero nel risultato.

**Questo processo si ripete su tutta l’immagine, generando una feature map.**

📘 Ogni filtro impara a riconoscere un tipo specifico di pattern:

* Un filtro può riconoscere i bordi orizzontali,

* Un altro può riconoscere i bordi verticali,

* Altri possono individuare curve o texture più complesse.

📊 Se applichi 8 filtri diversi, otterrai 8 feature map, ciascuna con informazioni differenti.

📷 Esempio visivo: come 2 filtri 3×3 generano 2 feature map da un’immagine 7×7×3

![Alt text](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*IGfVdsOnPl6MIwMO4V33IQ.png)

## 🔹 2. Stride

Lo stride indica di quanti pixel il filtro avanza a ogni passo (sia orizzontale che verticale).

* Stride = 1 → il filtro si muove di 1 pixel per volta → output grande (massimo dettaglio).

* Stride = 2 → il filtro si muove di 2 pixel per volta → output più piccolo (riduzione dimensionale).

📷 Esempio visivo: kernel 2×2 con stride=2 su immagine 5×5
![Alt text](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*DYi9mm55THoCH_-vqWXd7A.png)

## 🔹 3. Padding

Il padding aggiunge dei “pixel fittizi” (spesso zero) intorno all’immagine di input per controllare la dimensione dell’output e preservare le informazioni ai bordi.

* Senza padding: l’immagine si “rimpicciolisce” dopo ogni convoluzione.

* Con padding: possiamo mantenere la stessa dimensione dell’immagine originale.

📷 Esempio visivo: aggiunta di padding=1 a un’immagine 3×3
![Alt text](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*mf0nv_ajmLn7-TdmdnxFlQ.png)

## 🔹 4. Feature Map

Il risultato della convoluzione è una nuova immagine detta feature map o activation map.
Ogni feature map rappresenta come un determinato filtro “vede” l’immagine originale.

📷 Esempio visivo: diverse feature map prodotte da filtri differenti su immagini di cifre MNIST
![Alt text](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*11AOltIb4osdgt8Dtc98Kw.png)

In [ ]:
# Creiamo una piccola immagine 2D di dimensione 6x6 inizialmente tutta a zero (pixel neri)
img = np.zeros((6, 6), dtype=np.float32)

# Impostiamo a 1.0 (pixel bianchi) un quadrato 2x2 al centro: righe 2-3 e colonne 2-3 (lo slice finale è esclusivo)
# Così otteniamo un pattern semplice su cui "vedere" l'effetto della convoluzione
img[2:4,2:4] = 1.0

# Definiamo un kernel (filtro) 3x3 di tipo  per il gradiente ORIZZONTALE o VERTICALE
# ATTENZIONE: questi pesi corrispondono al  Gx (o a un suo segno invertito),
# cioè rilevano il GRADIENTE LUNGO X → evidenziano BORDI VERTICALI (cambiamenti sinistra↔destra).
# Per bordi verticali si usano valori con colonne [+1, 0, -1] (righe pesate 1,2,1).
# Intuizione: la colonna di sinistra è positiva (+1, +2, +1), quella di destra negativa (-1, -2, -1).
# Se a sinistra della finestra ho intensità più alte che a destra, la somma sarà positiva (bordi "in un verso");
# invertendo il segno si ribalta la direzione del contrasto (chiaro→scuro vs scuro→chiaro).
kernel = np.array([[ 1,  0, -1],
                   [ 2,  0, -2],
                   [ 1,  0, -1]], dtype=np.float32)


def conv2d_naive(image, kernel, stride=1, padding=0):
    """
    Implementazione "naive" della convoluzione 2D usando NumPy.
    NOTA: come nella maggior parte dei framework deep learning,
    qui eseguiamo in realtà una CORRELAZIONE (non ruotiamo il kernel di 180°).
    Questo è ciò che fanno anche PyTorch e TensorFlow nelle conv2d.
    """
    # Applichiamo il padding (se richiesto) aggiungendo bordi di zeri attorno all'immagine.
    # Il padding è utile per: (1) non "perdere" informazione ai bordi; (2) controllare la dimensione dell'output

    if padding > 0:
      image = np.pad(
          image,
          ((padding,padding),(padding,padding)),# (alto,basso),(sinistra)
          mode= "constant",
          constant_values=0
          )

      # H, W: altezza e larghezza dell'immagine (DOPO il padding, se applicato)
    H,W = image.shape

    # kH, kW: altezza e larghezza del kernel
    kH,kW = kernel.shape

    # Calcoliamo la dimensione spaziale dell'output in base a formula classica:
    # out = floor((in - kernel) / stride) + 1   (qui "in" è già H o W post-padding)
    #print(f"H = {H} & kh = {kH} & W = {W} & kW ={kW}")

    outH = (H - kH) // stride + 1
    outW = (W - kW) // stride + 1

    # Inizializziamo la mappa di attivazione (feature map) di output a zeri
    out = np.zeros((outH,outW),dtype=np.float32)

    # Doppio ciclo per scorrere il kernel su ogni posizione valida dell'immagine
    for i in range(outH):
      for j in range(outW):
        patch = image[i*stride:i*stride + kH,
                      j*stride:j*stride + kW]

        out[i,j] = np.sum(patch * kernel)


    # Ritorniamo la feature map risultante
    return out

# Convoluzione SENZA padding e con stride=1
# L'output si restringe (perché il kernel "non entra"0  sui bordi): da 6x6 con kernel 3x3 ottieni 4x4
out_no_pad = conv2d_naive(img,kernel,stride=1,padding=0)

# Convoluzione CON padding=1 e stride=1
# Il padding "allarga" l'immagine a 8x8 prima della conv; con kernel 3x3 e stride=1 l'output torna 6x6.
# (Manteniamo la stessa dimensione spaziale dell'input "originale": comportamento detto "same")
out_pad_1 = conv2d_naive(img,kernel,stride=1,padding=1)

# Convoluzione CON padding=1 ma stride=2
# Lo stride=2 "salta" una cella a ogni passo → output più piccolo (downsampling): da 6x6 ottieni 3x3
out_pad_2 = conv2d_naive(img,kernel,stride=2,padding=1)

# Stampiamo le forme per verificare visivamente gli effetti di padding e stride
print("Forma input",img.shape)
print("Senza padding,stride=1",out_no_pad.shape)
print("Padding=1,stride=1",out_pad_1.shape)
print("Padding=1, stride=2",out_pad_2.shape)

# Visualizziamo l'immagine di partenza (6x6) in scala di grigi
plt.figure()
plt.imshow(img,cmap ='gray')
plt.title("Immagine iniziale (6x6)")
plt.axis('off')
plt.show()


# Visualizziamo l'output della convoluzione con padding=1 e stride=1 (stessa dimensione dell'input)
# Qui ci aspettiamo valori alti in corrispondenza di bordi VERTICALI del quadrato bianco

plt.figure()
plt.imshow(out_no_pad,cmap ='gray')
plt.title("Immagine senza padding")
plt.axis('off')
plt.show()

plt.figure()
plt.imshow(out_pad_1,cmap ='gray')
plt.title("Immagine padding 1")
plt.axis('off')
plt.show()

plt.figure()
plt.imshow(out_pad_2,cmap ='gray')
plt.title("Immagine padding 2")
plt.axis('off')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# ==========================
# 1. Creiamo immagine e kernel
# ==========================
img = np.zeros((6, 6), dtype=np.float32)
img[2:4, 2:4] = 1.0  # piccolo quadrato bianco

# kernel = np.array([[ 1,  0, -1],
#                    [ 2,  0, -2],
#                    [ 1,  0, -1]], dtype=np.float32)

kernel = np.array([[ 1, 2, 1],
                   [ 0,  0, 0],
                   [ -1,  -2, -1]], dtype=np.float32)

stride = 1
padding = 1

# Applichiamo padding
padded_img = np.pad(img, ((padding, padding), (padding, padding)), mode='constant')
H, W = padded_img.shape
kH, kW = kernel.shape
outH = (H - kH)//stride + 1
outW = (W - kW)//stride + 1
out = np.zeros((outH, outW), dtype=np.float32)

# ==========================
# 2. Setup figura animata
# ==========================
plt.rcParams["animation.html"] = "jshtml"

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.set_title("Input + Kernel (con valori)")
ax2.set_title("Feature Map (valori aggiornati)")

im1 = ax1.imshow(padded_img, cmap='gray', vmin=0, vmax=1)
im2 = ax2.imshow(out, cmap='gray', vmin=-4, vmax=4)

# Rettangolo che rappresenta il kernel
rect = plt.Rectangle((0, 0), kW-1, kH-1, edgecolor='red', facecolor='none', lw=2)
ax1.add_patch(rect)

for ax in (ax1, ax2):
    ax.set_xticks([])
    ax.set_yticks([])

# Testo dinamico
text_kernel = fig.text(0.05, 0.02, "", fontsize=12, color='darkred')
text_value = fig.text(0.55, 0.02, "", fontsize=12, color='darkblue')

# Numeri della feature map
texts_out = [[ax2.text(j, i, "", ha="center", va="center", color="white", fontsize=10)
              for j in range(outW)] for i in range(outH)]

# Numeri del kernel che verranno visualizzati dentro il quadrato rosso
texts_kernel = [[ax1.text(0, 0, "", ha="center", va="center", color="yellow", fontsize=10)
                 for _ in range(kW)] for _ in range(kH)]

# ==========================
# 3. Funzione di aggiornamento
# ==========================
def update(frame):
    i = frame // outW
    j = frame % outW

    # estrai patch corrente
    patch = padded_img[i*stride : i*stride + kH,
                       j*stride : j*stride + kW]

    value = np.sum(patch * kernel)
    out[i, j] = value

    # aggiorna rettangolo
    rect.set_xy((j, i))
    im2.set_data(out)

    # aggiorna testi dinamici
    text_kernel.set_text(f"🧭 Calcolando output[{i},{j}]")
    text_value.set_text(f"📈 Σ(patch × kernel) = {value:.2f}")

    # aggiorna numeri sulla feature map
    for y in range(outH):
        for x in range(outW):
            texts_out[y][x].set_text(f"{out[y,x]:.1f}" if out[y,x] != 0 else "")

    # aggiorna numeri del kernel (centrati nella posizione corrente)
    for ki in range(kH):
        for kj in range(kW):
            x_pos = j + kj
            y_pos = i + ki
            texts_kernel[ki][kj].set_position((x_pos, y_pos))
            texts_kernel[ki][kj].set_text(f"{int(kernel[ki, kj])}")

    return [im1, im2, rect, text_kernel, text_value] + sum(texts_out, []) + sum(texts_kernel, [])

# ==========================
# 4. Animazione
# ==========================
ani = animation.FuncAnimation(
    fig, update, frames=outH*outW, interval=1000, blit=False, repeat=True
)

HTML(ani.to_jshtml())


# 🧭 Recap: cosa abbiamo fatto finora

Nel blocco precedente abbiamo:

1. **Creato un’immagine artificiale 6×6**, quasi tutta nera, con un piccolo quadrato bianco al centro.
→ Serve per osservare visivamente come i filtri rispondono ai bordi.

2. **Applicato una convoluzione 2D con un kernel(3×3)** che misura le variazioni di intensità orizzontali.

Il nostro kernel:

* colonne di sinistra positive (+1, +2, +1)

* colonne di destra negative (−1, −2, −1)
→ rileva bordi verticali, cioè transizioni nero ↔ bianco.

Visto l’effetto di padding e stride:

* Padding=0 → l’immagine “si restringe”

* Padding=1 → mantiene la stessa dimensione

* Stride=2 → dimezza l’output (downsampling)

Visualizzato l’output della convoluzione:

* Valori positivi (bianco) → bordo da scuro → chiaro

* Valori negativi (nero) → bordo da chiaro → scuro

* Valori vicini a 0 → regioni uniformi senza bordi

👉 In pratica, l’output rappresenta dove il filtro “vede” un bordo verticale.
Ogni valore della feature map è la risposta del filtro in quella posizione.

## 🧩 Passaggio successivo — Pooling

Ora applichiamo un pooling sull’output della convoluzione per:

1. ridurre la dimensione della feature map,

2. preservare le caratteristiche principali,

3. rendere il modello più robusto a piccoli spostamenti.

Esistono due tipi principali di pooling:

* **Max Pooling:** prende il valore massimo di ogni blocco (conserva le attivazioni più forti)

* **Average Pooling:** prende la media di ogni blocco (effetto di “lisciatura”)

Visivamente ⬇️

![Alt text](https://miro.medium.com/v2/resize:fit:640/format:webp/1*sIf_zGTXgvTEDkpeP8jvxg.jpeg)




In [ ]:
x = torch.tensor([[1],[2],[3]])
print(f"Tensore iniziale = {x} & shape = {x.shape}")

x_squeezed = torch.squeeze(x)
print(f"x_squezzed =  {x_squeezed} & shape = {x_squeezed.shape}")

x_unsqeezed = torch.unsqueeze(x_squeezed,dim=1)
print(f"x_unsqueezed = {x_unsqeezed} & shape = {x_unsqeezed.shape}")

x2 = torch.arange(1,9)
print(f"x2 = {x2} & shape = {x2.shape}")

x2_view = x2.view(2,4)
print(f"x2_view = {x2_view} & shape = {x2_view.shape}")

x2_reshape = x2.reshape(4,2)
print(f"x2_reshape = {x2_reshape} & shape = {x2_reshape.shape}")

x3 = torch.randn(2,3,4)
print(f"x3 = {x3} & shape = {x3.shape}")

x3_T = x3.transpose(0,1)
print(f"x3_t = {x3_T} & shape = {x3_T.shape}")

In [ ]:

# Convertiamo l'output della convoluzione (out_pad_1, un array NumPy 6x6) in un TENSORE PyTorch.
# Le reti convoluzionali in PyTorch si aspettano input con forma:
# (N, C, H, W)
# dove:
#   N = numero di esempi (batch size)
#   C = numero di canali (es. 1 per immagini in scala di grigi, 3 per RGB)
#   H = altezza (height)
#   W = larghezza (width)

# out_pad_1 ha shape (6, 6) → solo H e W.
# Per renderlo compatibile, aggiungiamo:
# - una dimensione per il batch (N=1)
# - una per il canale (C=1)
# Risultato: (1, 1, 6, 6)
feature_map = torch.tensor(out_pad_1,dtype=torch.float32).unsqueeze(0).unsqueeze(0)
print(f"feaure_map = {feature_map.shape}")

# Definiamo i due tipi di pooling:
# MaxPool2d: seleziona il valore massimo in ogni finestra (kernel 2x2)
# AvgPool2d: calcola la media dei valori nella stessa finestra
# Lo stride=2 significa che ogni finestra "salta" di 2 pixel → dimezza H e W (6x6 → 3x3)
maxpool = nn.MaxPool2d(kernel_size=2,stride=2)
avgpool = nn.AvgPool2d(kernel_size=2,stride=2)

# Applichiamo il pooling alla feature map
max_pooled = maxpool(feature_map)
avg_pooled = avgpool(feature_map)

# Stampiamo le forme per verificare la riduzione spaziale
print(f"Max pooling = {max_pooled.shape}")
print(f"Avg pooling = {avg_pooled.shape}")

# Visualizziamo i risultati a confronto:
# - Feature map originale (output della convoluzione)
# - Max pooling (3x3): mantiene solo i valori massimi locali
# - Average pooling (3x3): calcola la media locale, smussando i dettagli
fig, axes = plt.subplots(1,3,figsize = (10,3))

axes[0].imshow(out_pad_1,cmap= 'gray')
axes[0].set_title("Feature map originale")

axes[1].imshow(max_pooled.squeeze(), cmap='gray')
axes[1].set_title("Max pooling")

axes[2].imshow(avg_pooled.squeeze(),cmap='gray')
axes[2].set_title("Avg Pooling")

for ax in axes:
  ax.axis('off')


plt.show()

# Rimuoviamo gli assi per una visualizzazione più pulita



# 📊 Cosa osserviamo

* Dopo il **Max Pooling**, l’immagine è più piccola (6×6 → 3×3) ma i bordi principali sono ancora ben evidenti.

* Dopo **l’Average Pooling**, le variazioni si “ammorbidiscono”: le intensità estreme vengono mediate.

✅ Conclusione

Il pooling serve per:

* ridurre la quantità di dati da elaborare nei layer successivi,

* mantenere solo le informazioni più significative,

* garantire invarianza a traslazioni locali (es. un bordo spostato di 1 pixel non cambia molto l’attivazione del blocco).

In una CNN reale, dopo ogni convoluzione si alternano tipicamente:

**Conv → ReLU → Pooling**

per costruire progressivamente rappresentazioni più compatte ma più ricche di significato.


## La tua prima CNN (MNIST)

Costruiremo una piccola CNN per **MNIST** (cifre 28×28 in scala di grigi).  
**Pipeline:**
1. **Transforms**: conversione a tensore e **normalizzazione** (media=0.1307, std=0.3081).  
2. **Dataset & DataLoader** per train/val/test.  
3. **Modello**: due blocchi conv → flatten → fully connected.  
4. **Training loop** con `CrossEntropyLoss` + `Adam`.  
5. **Accuratezza** e barra `tqdm` per il progresso.  
6. **Grafici** di loss e accuracy.


In [ ]:
# 3.1 Trasformazioni (normalizzazione consigliata per MNIST)
# Compose concatena più trasformazioni da applicare in sequenza a ogni immagine.
# ToTensor: converte un'immagine PIL/array [0..255] in un tensore float [0..1] con shape (C,H,W).
# Normalize: per ogni canale applica (x - mean) / std. Per MNIST (grayscale) c'è un solo canale.
#            mean=0.1307 e std=0.3081 sono statistiche standard del dataset MNIST.
transform_mnist = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,),(0.3081,))
    ]
)

# 3.2 Dataset (con fallback)
# Directory locale dove scaricare o cercare i file del dataset.
data_root = './data'

# Proviamo a caricare/scaricare MNIST:
# - train=True: split di addestramento (60k immagini)
# - train=False: split di test (10k immagini)
# - transform=transform_mnist: applica le trasformazioni definite sopra
# - download=True: scarica se non è già presente nella cartella data_root
try:
  train_full = datasets.MNIST(root=data_root,train=True,transform=transform_mnist,download=True)
  test_ds = datasets.MNIST(root=data_root,train=False,transform=transform_mnist,download=True)
except Exception as e:
  print(f"impossibile scaricare MNIST",str(e))

# 3.3 Split train/val
# Percentuale del training set da usare come validazione (10%).
val_ratio = 0.1
val_size = int(len(train_full) * val_ratio)
train_size = len(train_full) - val_size
train_ds,val_ds = random_split(train_full,[train_size,val_size])
len(train_full),len(train_ds),len(val_ds),len(test_ds)

In [ ]:

# 3.4 DataLoader
# Dimensione dei batch (128 è un buon compromesso per MNIST; puoi aumentarla con GPU più capienti).
batch_size = 128

train_loader = DataLoader(train_ds,batch_size=batch_size,shuffle=True,num_workers=2,
                          pin_memory=True if device.type == 'cuda' else False
                          )
val_loader = DataLoader(val_ds,batch_size=batch_size, shuffle=True,num_workers=2,
                        pin_memory=True if device.type == 'cuda' else False
                        )
test_loader = DataLoader(test_ds,batch_size=batch_size, shuffle=True,num_workers=2,
                        pin_memory=True if device.type == 'cuda' else False
                        )


In [ ]:
# 3.5 Visualizziamo alcune immagini con le etichette
def show_batch(dl, n=5):
    # Stampa la lunghezza del DataLoader, ovvero il numero totale di batch.
    # Esempio: se il dataset ha 54.000 immagini e batch_size=128 → ci saranno circa 422 batch.
    print(len(dl))

    # next(iter(dl)) estrae il PRIMO batch dal DataLoader.
    # xb = immagini (batch di tensori)
    # yb = etichette (batch di target numerici)
    xb, yb = next(iter(dl))

    # Stampa le shape dei tensori per capire come sono strutturati.
    print(xb.shape)  # (batch_size, canali, altezza, larghezza)
    print(yb.shape)  # (batch_size,)

    #print(xb)  # (batch_size, canali, altezza, larghezza)
    #print(yb)  # (batch_size,)

    # Seleziona solo i primi n campioni del batch (es. 5 immagini)
    xb, yb = xb[:n], yb[:n]

    # Sposta i dati su CPU e li converte in array NumPy per poterli plottare con Matplotlib.
    xb = xb.cpu().numpy()

    # Crea una nuova figura.
    plt.figure()

    # cols = numero di colonne nel layout dei subplot (una riga con n immagini)
    cols = n
    for i in range(n):
        # Crea una subplot 1×n, posizione i+1
        plt.subplot(1, cols, i+1)

        # xb[i][0]: seleziona la i-esima immagine e il primo canale (0, perché è in bianco e nero)
        plt.imshow(xb[i][0], cmap='gray')

        # Mostra il numero della classe come titolo
        plt.title(f"Label: {int(yb[i])}")

        # Nasconde gli assi per una visualizzazione più pulita
        plt.axis('off')

    # Mostra tutte le immagini insieme
    plt.show()

# Esegui la funzione per visualizzare 5 immagini dal training set
show_batch(train_loader, n=5)


In [ ]:
# 3.6 Definiamo la CNN
class CNN(nn.Module):
  def __init__(self,dropout:0,use_batchnorm=False):

    super().__init__()

    c1,c2 = 5,10

    layers = []
    #(28,28)
    layers += [nn.Conv2d(1,c1,kernel_size=3,padding=1),
               nn.ReLU(inplace=True)]

    if use_batchnorm:
      layers += [nn.BatchNorm2d(c1)]

    layers += [nn.MaxPool2d(2,2)] # 28 -> 14

    layers += [nn.Conv2d(c1,c2,kernel_size=3,padding=1), # (14,14)
               nn.ReLU(inplace=True)]

    if use_batchnorm:
      layers += [nn.BatchNorm2d(c2)]

    layers += [nn.MaxPool2d(2,2)] # 14 -> 7

    self.features = nn.Sequential(*layers)

    self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

    self.classifier = nn.Linear(7 * 7 *c2 ,10)

  def forward(self,x):
    x = self.features(x)
    x = torch.flatten(x,1)
    x = self.dropout(x)
    x= self.classifier(x)
    return x

def count_params(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = CNN(dropout=0,use_batchnorm=False).to(device)

print(model)

print("Parametri addestrabili",count_params(model))



In [ ]:
# Calcola l'accuratezza a partire dai logits (uscita del modello) e dalle label vere y
def accuracy(logits,y):
  preds = logits.argmax(dim=1)
  """
  logits = [0.9,0.1,0,0,0,0]
  preds = 0

  tensor([0])
  """


  return (preds == y).float().mean().item()


# Esegue UNA epoca di training (forward + backward + update pesi) e restituisce loss/accuracy medie sull'epoca
def train_epochs(model,loader,optimizer,criterior):
  model.train()
  running_loss,running_acc = 0.0,0.0

  for xb,yb in tqdm(loader,desc='Train',leave=False):

    xb,yb = xb.to(device),yb.to(device)

    optimizer.zero_grad()

    logits = model(xb)

    loss = criterior(logits,yb)

    loss.backward()

    optimizer.step()

    running_loss += loss.item() * xb.size(0)

    running_acc += accuracy(logits,yb) * xb.size(0)

  n = len(loader.dataset)

  return running_loss / n , running_acc / n

# Valutazione (validazione/test): NESSUN gradiente, NESSUN update pesi
@torch.no_grad()
def evaluate(model,loader,criterior):
  model.eval()
  running_loss, running_acc = 0.0,0.0

  for xb,yb in tqdm(loader,desc='val',leave= False):
    xb,yb = xb.to(device),yb.to(device)
    logits = model(xb)
    loss = criterior(logits,yb)

    running_loss += loss.item() * xb.size(0)

    running_acc += accuracy(logits,yb) * xb.size(0)

  n = len(loader.dataset)

  return running_loss / n , running_acc / n


In [ ]:
# 3.8 Addestriamo!
epochs = 3
criterior = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001,weight_decay=0)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=2,gamma = 0.5)

history = {"train_loss":[],"val_loss":[],"train_acc":[],"val_acc":[]}

for epoch in range(1,epochs + 1):
  print(f"Epochs {epoch}/{epochs}")
  tl,ta = train_epochs(model,train_loader,optimizer,criterior)
  vl,va = evaluate(model,val_loader,criterior)
  scheduler.step()

  history["train_loss"].append(tl)
  history["train_acc"].append(ta)
  history["val_loss"].append(vl)
  history["val_acc"].append(va)

  print(f"loss_train = {tl:.4f} acc_train = {ta:.4f} | loss_val = {vl:.4f} acc_val = {va:.4f}")



In [ ]:

# 3.9 Grafici delle curve di apprendimento (matplotlib; un grafico per figura; nessuno stile colore specifico)



In [ ]:

# 3.10 Valutazione sul test set + alcune predizioni




## 4. Regolarizzazione e miglioramenti

**Perché servono?** Per ridurre **overfitting** e rendere l'addestramento più stabile.

- **Dropout:** azzera casualmente alcune attivazioni durante il training (es. `p=0.3`).  
- **Weight decay (L2):** penalizza pesi grandi tramite l'ottimizzatore (es. `weight_decay=1e-4`).  
- **Batch Normalization:** normalizza le attivazioni per velocizzare/stabilizzare il training.  
- **Scheduler LR:** regola il learning rate durante l'addestramento (es. `StepLR`, `OneCycleLR`).

Sotto attiviamo **dropout + batch norm** e facciamo un breve training dimostrativo.



## 4bis. Confusion Matrix ed Early Stopping

In questa sezione impariamo due strumenti fondamentali per migliorare l’analisi e la robustezza del modello:

1. **Confusion Matrix (Matrice di confusione)**  
   Serve per capire **quali classi il modello confonde** tra loro.  
   Ogni cella mostra quante volte una classe reale è stata predetta come un’altra classe.

   - La diagonale rappresenta le **predizioni corrette**.  
   - Le celle fuori diagonale mostrano gli **errori di classificazione**.

2. **Early Stopping (Interruzione anticipata)**  
   Serve a fermare l’addestramento **prima** che il modello inizi a sovradattarsi (*overfitting*).  
   Se la **loss di validazione** non migliora dopo un certo numero di epoche (pazienza), si interrompe il training.


In [ ]:
# Importiamo due utility da scikit-learn:
# - confusion_matrix: calcola la matrice di confusione a partire da etichette vere e predette
# - ConfusionMatrixDisplay: oggetto comodo per visualizzare la matrice con Matplotlib


# --- Calcolo e visualizzazione della matrice di confusione sul TEST set ---

                                         # Mostriamo la figura


In [ ]:
# Early Stopping semplice: si interrompe se la loss di validazione non migliora per 'patience' epoche consecutive




### Visualizzazione delle curve di Early Stopping

Nei grafici seguenti vediamo come si comportano **loss** e **accuratezza** nel tempo:  
- le curve blu rappresentano l’andamento del **training**;  
- le curve arancioni rappresentano l’**insieme di validazione**;  
- il **punto rosso** indica il momento in cui l’early stopping ha interrotto l’addestramento.

Osservando i grafici puoi capire quando il modello ha smesso di migliorare sulla validazione, pur continuando a migliorare (overfitting) sul training.


In [ ]:

# Troviamo l'epoca di stop (ultima epoca effettiva eseguita)
stopped_epoch = len(hist_es["val_loss"]) - 1

plt.figure()
plt.plot(hist_es["train_loss"], label="loss_train")
plt.plot(hist_es["val_loss"], label="loss_val")
plt.scatter(stopped_epoch, hist_es["val_loss"][stopped_epoch], color="red", label="stop")
plt.legend(); plt.xlabel("epoca"); plt.ylabel("loss")
plt.title("Early Stopping - Andamento della loss")
plt.show()

plt.figure()
plt.plot(hist_es["train_acc"], label="acc_train")
plt.plot(hist_es["val_acc"], label="acc_val")
plt.scatter(stopped_epoch, hist_es["val_acc"][stopped_epoch], color="red", label="stop")
plt.legend(); plt.xlabel("epoca"); plt.ylabel("accuratezza")
plt.title("Early Stopping - Andamento dell'accuratezza")
plt.show()



## 5. Salvataggio e caricamento del modello

Usa `state_dict` per salvare solo i pesi:



## 6. (Opzionale) Assaggio di transfer learning (CIFAR-10)

**Idea:** partire da un modello pre-addestrato su un grande dataset (es. ImageNet) e adattarlo al tuo compito.

Useremo **ResNet18** da `torchvision.models`. Se non è possibile scaricare il dataset, ricorriamo a `FakeData` (3×32×32, ridimensionate a 224×224).  
È solo un assaggio: per risultati solidi servono più epoche e un setup più curato.


In [ ]:
# Importiamo la ResNet18 e i pesi pre-addestrati disponibili in torchvision


# ----------------------------
# 1️⃣ Definizione delle trasformazioni
# ----------------------------

])

# ----------------------------
# 2️⃣ Caricamento dataset CIFAR-10 (con fallback)
# ----------------------------


# ----------------------------
# 3️⃣ Creazione DataLoader
# ----------------------------
# batch_size=64 → 64 immagini per batch
# shuffle=True nel training → mescola i dati a ogni epoca per migliorare la generalizzazione
# pin_memory=True (solo su GPU): migliora le prestazioni delle copie CPU→GPU


# ----------------------------
# 4️⃣ Caricamento modello pre-addestrato
# ----------------------------
# Otteniamo i pesi predefiniti di ResNet18 (pretrained su ImageNet)


# Carichiamo la rete con quei pesi


# ----------------------------
# 5️⃣ Congelamento dei layer di feature extraction
# ----------------------------
# In questo modo NON aggiorniamo i pesi delle convoluzioni già apprese su ImageNet
# (evitiamo di distruggere conoscenze generali come rilevatori di bordi, texture, forme)


# ----------------------------
# 6️⃣ Sostituzione della testa di classificazione
# ----------------------------
# ResNet18 pre-addestrata ha un layer finale (fc) con 1000 output (per ImageNet)
# Lo sostituiamo con un layer fully connected per 10 classi (CIFAR-10)


# Spostiamo tutto il modello sul device (CPU o GPU)


# ----------------------------
# 7️⃣ Ottimizzatore e loss function
# ----------------------------
# Addestriamo SOLO la nuova testa (gli altri layer restano congelati)

# Loss per classificazione multi-classe

# ----------------------------
# 8️⃣ Mini addestramento dimostrativo
# ----------------------------
# Solo 1 epoca per mostrare come funziona il transfer learning



In [ ]:
# Classi di CIFAR-10
cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                 'dog', 'frog', 'horse', 'ship', 'truck']

def predict_and_show(model, loader, n=5):
    """
    Mostra n immagini del loader con la predizione del modello.
    """
    model.eval()  # disattiva dropout e batchnorm in modalità training
    xb, yb = next(iter(loader))          # prende il primo batch
    xb, yb = xb.to(device), yb.to(device)

    with torch.no_grad():                # no gradient → più veloce, meno memoria
        preds = model(xb).argmax(1)      # indice della classe con logit più alto

    xb = xb.cpu().permute(0, 2, 3, 1)    # da (N,C,H,W) → (N,H,W,C) per Matplotlib
    xb = xb * torch.tensor((0.229, 0.224, 0.225)) + torch.tensor((0.485, 0.456, 0.406))
    xb = torch.clamp(xb, 0, 1)           # de-normalizzazione per visualizzare correttamente

    plt.figure(figsize=(12, 4))
    for i in range(n):
        plt.subplot(1, n, i+1)
        plt.imshow(xb[i])
        plt.title(f"True: {cifar_classes[yb[i]]}\nPred: {cifar_classes[preds[i]]}",
                  fontsize=10)
        plt.axis('off')
    plt.show()

# 🔍 Eseguiamo la funzione di predizione
predict_and_show(backbone, cifar_test_loader, n=5)
